<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [1]:
! pip install pyspark py4j
! pip install pyodbc
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**Importing modules and pandas library that will be used later.**

In [2]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import pyodbc
import findspark

findspark.init()

In [3]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

23/07/06 14:18:41 WARN Utils: Your hostname, Linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/06 14:18:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 14:18:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Gooogle Colab Version

In [192]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True) 

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [4]:
fille =r'../notebooks/Final/Parquet/HistoricalPrincing.parquet'

Histprice = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [5]:
# Balance table
Histprice.createOrReplaceTempView("Histprice")

In [6]:
Histprice.show()

+-----------------+-----------------+-----------------+-----------------+---------+--------+----------+
|         Abertura|             Alta|            Baixa|            close|   Volume|  symbol|      date|
+-----------------+-----------------+-----------------+-----------------+---------+--------+----------+
|5.699999809265137|5.739999771118164|5.610000133514404|5.630000114440918| 800900.0|ZAMP3.SA|2022-07-04|
|5.599999904632568|5.809999942779541|5.449999809265137|5.809999942779541|2219100.0|ZAMP3.SA|2022-07-05|
|5.800000190734863| 5.96999979019165|5.710000038146973|5.880000114440918|1151300.0|ZAMP3.SA|2022-07-06|
|5.960000038146973|6.170000076293945|5.909999847412109|5.989999771118164|1614300.0|ZAMP3.SA|2022-07-07|
|6.039999961853027|6.130000114440918|5.869999885559082|5.949999809265137| 528600.0|ZAMP3.SA|2022-07-08|
|5.889999866485596|5.980000019073486|5.710000038146973| 5.71999979019165| 803000.0|ZAMP3.SA|2022-07-11|
|5.739999771118164|5.920000076293945|5.650000095367432|5.7699999

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [20]:
Table = spark.sql("""
SELECT  REGEXP_REPLACE(symbol, '\\.SA$', '') AS Ticker,
        ROUND(Abertura,2) as Abertura,
        ROUND(Alta,2)  AS Alta,
        ROUND(Baixa,2) AS Baixa,
        ROUND(close,2) AS Fechamento,
        ROUND(Volume,2) AS Volume,
        CAST(date AS STRING) AS Data
FROM Histprice
""")

In [21]:
Table.show()

+------+--------+----+-----+----------+---------+----------+
|Ticker|Abertura|Alta|Baixa|Fechamento|   Volume|      Data|
+------+--------+----+-----+----------+---------+----------+
| ZAMP3|     5.7|5.74| 5.61|      5.63| 800900.0|2022-07-04|
| ZAMP3|     5.6|5.81| 5.45|      5.81|2219100.0|2022-07-05|
| ZAMP3|     5.8|5.97| 5.71|      5.88|1151300.0|2022-07-06|
| ZAMP3|    5.96|6.17| 5.91|      5.99|1614300.0|2022-07-07|
| ZAMP3|    6.04|6.13| 5.87|      5.95| 528600.0|2022-07-08|
| ZAMP3|    5.89|5.98| 5.71|      5.72| 803000.0|2022-07-11|
| ZAMP3|    5.74|5.92| 5.65|      5.77|1001600.0|2022-07-12|
| ZAMP3|    5.74|5.88| 5.69|      5.73|1311000.0|2022-07-13|
| ZAMP3|     5.7|5.83| 5.62|      5.78|1441100.0|2022-07-14|
| ZAMP3|     5.8|6.19| 5.74|      6.09|2020000.0|2022-07-15|
| ZAMP3|    6.08|6.08| 5.78|       5.8|1325000.0|2022-07-18|
| ZAMP3|    5.83|5.85| 5.58|      5.63|1922100.0|2022-07-19|
| ZAMP3|    5.58| 5.9| 5.58|       5.8|2687700.0|2022-07-20|
| ZAMP3|    5.75|5.86| 5

In [22]:
dftable = Table.toPandas()

In [23]:
dftable.to_excel("../notebooks/Final/HistoryPrice.xlsx",
                 sheet_name = "Historical_Price")